In [3]:
# --- Celda 1: Importaciones ---

import os
import sys
import pandas as pd
import numpy as np
# Bibliotecas para el Dashboard
from jupyter_dash import JupyterDash
import dash
from dash import Dash
from dash import dcc, html, Input, Output, State
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objects as go
from dashboard_modules.callbacks.cb_tab1_monitoring_stations import register_tab1_callbacks

print("Bibliotecas importadas exitosamente.")

Bibliotecas importadas exitosamente.


In [4]:
# --- Celda 2: Configuración de Rutas y Carga de Datos ---

# Configurar la ruta raíz del proyecto para poder importar módulos
# Asumiendo que este notebook está en 'analisis-compcientifica/notebooks/'
# --- Celda 2: Configuración de Rutas y Carga de Datos ---
# Asumiendo que este notebook está en 'analisis-compcientifica/notebooks/'
# Sube un nivel para llegar a la raíz del proyecto ('analisis-compcientifica/')
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Añadir la raíz del proyecto al path de Python si no está ya
if project_root not in sys.path:
    print(f"Añadiendo ruta del proyecto al path: {project_root}")
    sys.path.append(project_root)
# --- FIN DEL BLOQUE IMPORTANTE ---

# Rutas a los archivos de datos procesados
path_emisiones = os.path.join(project_root, 'data', 'processed', 'emission_sources_processed.pkl')
# ... (resto de tu código de carga de datos) ...
# Rutas a los archivos de datos procesados
path_emisiones = os.path.join(project_root, 'data', 'processed', 'emission_sources_processed.pkl')
path_estaciones = os.path.join(project_root, 'data', 'processed', 'air_quality_stations_processed.pkl')
path_poblacion = os.path.join(project_root, 'data', 'processed', 'population_processed.pkl')

# Cargar los DataFrames
try:
    df_emisiones = pd.read_pickle(path_emisiones)
    df_estaciones = pd.read_pickle(path_estaciones)
    df_poblacion = pd.read_pickle(path_poblacion)
    
    # --- AÑADIR ESTA LÍNEA PARA CORREGIR EL ERROR ---
    # Asegurar que la columna 'timestamp' sea del tipo datetime
    df_estaciones['timestamp'] = pd.to_datetime(df_estaciones['timestamp'], errors='coerce')
    
    print("\n¡Los 3 archivos de datos procesados se cargaron exitosamente!")
except FileNotFoundError as e:
    print(f"\nERROR: No se encontró uno de los archivos .pkl. Asegúrate de que los scripts ETL se hayan ejecutado correctamente.")
    print(f"Detalle del error: {e}")
except Exception as e:
    print(f"\nOcurrió un error al cargar los archivos de datos: {e}")


# --- Verificación Rápida de los Datos Cargados ---
if 'df_emisiones' in locals():
    print("\n--- Datos de Emisiones (RECT) ---")
    print(f"Forma: {df_emisiones.shape}")
    display(df_emisiones.head(3))

if 'df_estaciones' in locals():
    print("\n--- Datos de Estaciones (AirQualitySM) ---")
    print(f"Forma: {df_estaciones.shape}")
    # Verificar el tipo de dato de la columna 'timestamp'
    print("Info de la columna 'timestamp':")
    df_estaciones[['timestamp']].info() 
    display(df_estaciones.head(3))

if 'df_poblacion' in locals():
    print("\n--- Datos de Población ---")
    print(f"Forma: {df_poblacion.shape}")
    display(df_poblacion.head(3))


¡Los 3 archivos de datos procesados se cargaron exitosamente!

--- Datos de Emisiones (RECT) ---
Forma: (5223049, 6)


,año,region,comuna,cantidad_toneladas,contaminante,tipo_fuente
0,2019,Antofagasta,Antofagasta,3.037223,Carbono Negro,Difusas
1,2019,Antofagasta,Mejillones,0.104612,Carbono Negro,Difusas
2,2019,Antofagasta,Sierra Gorda,0.012500,Carbono Negro,Difusas



--- Datos de Estaciones (AirQualitySM) ---
Forma: (132913, 9)
Info de la columna 'timestamp':
<class 'pandas.core.frame.DataFrame'>
Index: 132913 entries, 3827 to 129328
Data columns (total 1 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   timestamp  132913 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 2.0 MB


,timestamp,region,estacion,pm25,pm10,o3,no2,co,so2
3827,2014-01-30,Arica,Arica,63.0,NaN,NaN,NaN,NaN,NaN
3828,2014-01-31,Arica,Arica,70.0,NaN,NaN,NaN,NaN,NaN
3829,2014-02-01,Arica,Arica,70.0,NaN,NaN,NaN,NaN,NaN



--- Datos de Población ---
Forma: (544, 4)


,codigo_region,region,año,poblacion
0,1,Tarapacá,2002,246265
1,1,Tarapacá,2003,251625
2,1,Tarapacá,2004,257394


In [5]:
# --- Celda 3: Creación del Layout Principal y Esqueleto de la App (Versión para Pestaña 1) ---

# Importar la función de layout que acabamos de crear
from dashboard_modules.layouts.tab1_monitoring_stations import create_layout_tab1

# 1. Inicializar la aplicación Dash con el modo para Jupyter
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Preparar listas para los filtros
# Asegúrate de que df_estaciones y df_poblacion estén cargados de la celda anterior
lista_regiones_disponibles = sorted(df_estaciones['region'].unique()) if 'df_estaciones' in locals() else []
lista_años_disponibles = sorted(df_estaciones['timestamp'].dt.year.unique()) if 'df_estaciones' in locals() else []


# 2. Definir el layout principal de la aplicación
app.layout = dbc.Container(
    [
        dbc.Row(
            dbc.Col(
                html.H1("Dashboard Interactivo: Calidad del Aire en Chile", 
                        className="text-center text-primary my-4"),
                width=12
            )
        ),
        
        dcc.Tabs(
            id="tabs-principales",
            value="tab-estaciones",
            children=[
                # Aquí usamos la función para crear y añadir el contenido de la Pestaña 1
                dcc.Tab(
                    label="Estaciones de Monitoreo", 
                    value="tab-estaciones",
                    children=create_layout_tab1(lista_regiones_disponibles, lista_años_disponibles)
                ),
                dcc.Tab(label="Fuentes de Contaminación", value="tab-fuentes", children=[html.Div("Contenido Pestaña 2")]),
                dcc.Tab(label="Correlación de Datos", value="tab-correlacion", children=[html.Div("Contenido Pestaña 3")]),
            ],
            colors={
                "border": "white",
                "primary": "#0d6efd",
                "background": "#f8f9fa"
            }
        ),
        
        # Este div ya no es necesario si ponemos el contenido directamente en el children de cada Tab
        # html.Div(id="contenido-pestañas", className="mt-4")
    ],
    fluid=True
)


print("El layout de la aplicación Dash ha sido definido, incluyendo el contenido de la Pestaña 1.")

El layout de la aplicación Dash ha sido definido, incluyendo el contenido de la Pestaña 1.


In [6]:
# --- Celda 4: Registrar Callbacks de la Aplicación ---

try:
    register_tab1_callbacks(app, df_estaciones, df_poblacion)
    print("Callbacks para la Pestaña 1 registrados exitosamente.")
except NameError as e:
    print(f"Error al registrar callbacks: Asegúrate de que los DataFrames (ej. df_estaciones) estén cargados. Detalle: {e}")
except Exception as e:
    print(f"Ocurrió un error inesperado al registrar los callbacks: {e}")

Callbacks para la Pestaña 1 registrados exitosamente.


In [7]:
app.run(mode='tab', port=8051) # Usamos un puerto diferente por si tienes otro dash corriendo


--- Callback de Estación Activado ---
Región recibida como input: 'Arica' (tipo: <class 'str'>)
  -> Estaciones encontradas para 'Arica': ['Arica']
  -> Opciones generadas para el dropdown: [{'label': 'Arica', 'value': 'Arica'}]
  -> Valor inicial seleccionado: 'Arica'
  -> Habilitando dropdown y devolviendo valores.
